In [4]:
import numpy as np
from numpy.linalg import matrix_power as mpow

In [5]:
#Grafo de prueba, A-B y A-C
A = np.array([[0,1,1],[1,0,0],[0,0,1]])
A

array([[0, 1, 1],
       [1, 0, 0],
       [0, 0, 1]])

In [83]:
#Primero enumeracion simple (grafos sin direccion)
#Las restricciones son un diccionario con las clases como llave,
#con sets indicando que elementos no estan permitidos
def enumeracion(A, restricciones, MAX, symbs):
    if MAX<0:
        MAX=np.inf;
        
    n = len(A)
    
    #Inicializamos una matriz para el proceso en zeros
    S = np.zeros((n,n))
    S[0] = [sum(a) for a in A]
    
    
    #Arreglo para checar los nodos restantes
    left = [i for i in range(n)]
    
    #La solucion es un diccionario con las clases como llaves
    soln = {}
    #Inicializamos los 
    for i in range(n):
        
        #Inicializamos maximo en -infinito e indice en -1
        m = -np.inf
        index = -1
        
        #Buscamos el maximo en los nodos restantes
        for j in left:
            if S[i,j] > m :
                m = S[i,j]
                index = j
        
        #Nuestra solucion es index, la quitamos de los nodos disponibles
        left.remove(index)
        
        #El siguiente renglon es el mismo que el pasado menos las conexiones de index
        if i-1<m:
            S[i+1] = S[i] - A[index]
        
        #Agregamos index a su clase, tomando en cuenta las restricciones puestas, intentando primero en la clase 0
        clase = 0
        flag = False
        
        #Busca la clase
        while not flag:
            #Verifica que la nodo pueda estar en esa clase
            try:
                claseProhibida=symbs[index] in restricciones[clase]
            except Exception:
                claseProhibida=False
            
            #Ve si la clase está llena
            try: 
                maxElementos=not len(soln[clase])<MAX
            except Exception:
                maxElementos=False
            
            if not claseProhibida and not maxElementos:
                #Agrega el nodo a la clase y agrega las restricciones correspondientes
                flag=True
                
                try:
                    soln[clase].append(symbs[index])
                except Exception:
                    soln[clase] = [symbs[index]]
                
                try:
                    restricciones[clase]|=set([symbs[x] for x in np.where(A[index]==1)[0]])
                except Exception:
                    restricciones[clase] = set([symbs[x] for x in np.where(A[index]==1)[0]])
            
            else:
                clase += 1
            
    return soln

In [84]:
print('Enumeracion Simple')
syms = input('Escriba todos los nodos, separados por comas: \n')

symbs = syms.replace(' ','').split(',')
n = len(symbs)
syms = { symbs[i] : i for i in range(n) }
A = np.zeros((n,n))

print('Escriba las conexiones, separando los elementos com comas:')

flag = True
while flag:
    s = input('Clan: \n')
    s = s.replace(' ' ,'').split(',')
    
    for i in s:
        for j in [k for k in s if k!=i]:
            A[syms[i],syms[j]] = 1
    flag = (input('Hay otra conexión? (S/N): \n') == 'S')

MAX = int(input ('Ingrese el número máximo de elementos por clase (si no hay ingrese un numero negativo)'))

restricciones = {}

while (input('Hay restricciones sobre los nodos? (S/N): \n') == 'S'):
    s = input('Restriccion con formato A >= n (clase de A mayor igual a n)\n')
    try:
        sym, index = s.replace(' ','').split('>=')
        index = int(index)
    except Exception:
        print('Restriccion invalida')
    
    for j in range(index):
        try:
            restricciones[j].add(sym)
        except Exception:
            restricciones[j] = set([sym])

Enumeracion Simple
Escriba todos los nodos, separados por comas: 
a,b,c,d,e,f
Escriba las conexiones, separando los elementos com comas:
Clan: 
a,c,b
Hay otra conexión? (S/N): 
N
Ingrese el número máximo de elementos por clase (si no hay ingrese un numero negativo)-3
Hay restricciones sobre los nodos? (S/N): 
S
Restriccion con formato A >= n (clase de A mayor igual a n)
a>=2
Hay restricciones sobre los nodos? (S/N): 
N


In [86]:
enumeracion(A,restricciones,MAX,symbs)

{0: ['b', 'd', 'e', 'f'], 1: ['c'], 2: ['a']}

In [7]:
#Super ineficiente, mayb cambiar por algo q calcule todo a la vez elevando la matriz hasta la n-esima potencia
def reachable(A,index):
    n = len(A)
    visited = set()
    tovisit = set(np.where(A[index] == 1)[0])
    
    while len(visited) < n and not tovisit == set():
        visited = visited | tovisit
        
        aux = set()
        for k in tovisit:
            aux = aux | set(np.where(A[k] == 1)[0])
        
        tovisit = aux - visited
    
    return visited

In [8]:
#Falta considerar restricciones, no pifa bien (?)
def enumeracionDirigida(A, restricciones = {}):
    n = len(A)
    
    #Armamos las conecciones en un diccionario con nodo como llave, destinos como elementos
    conecciones = {}
    for i in range(n):
        conecciones[i] = reachable(A, i)
    
    print(conecciones)
    #Inicializamos una matriz para el proceso en zeros
    S = np.zeros((n,n))
    S[0] = [len(c) for c in conecciones.values()]
    
    
    #Arreglo para checar los nodos restantes
    left = [i for i in range(n)]
    
    #La solucion es un diccionario con las clases como llaves
    soln = {}
    #Inicializamos los 
    for i in range(n):
        
        #Inicializamos maximo en -infinito e indice en -1
        m = -np.inf
        index = -1
        
        #Buscamos el maximo en los nodos restantes
        for j in left:
            if S[i,j] > m :
                m = S[i,j]
                index = j
        
        #Nuesta solucion es index, la quitamos de los nodos disponibles
        left.remove(index)
        
        #El singuiente renglon es el mmismo que el pasado menos las conecciones de index
        if i-1<m:
            S[i+1] = S[i] - [1 if k in conecciones[index] else 0 for k in range(n)]
        
        #Agregamos index a su clase, tomando en cuenta las restricciones puestas, intentando primero en la clase 0
        clase = 0
        flag = False
        
        #Busca la clase
        while not flag:
            try:
                if index not in restricciones[clase]:
                    flag = True
                    restricciones[clase] = restricciones[clase] | conecciones[index]
                    
                else:
                    #Restringir todo lo q se alcanza desde el nodo en esa clase? (i.e tiene q ser despues si lo alcanzas desde ese nodo?)
                    clase += 1
                    
            except Exception:
                flag = True
                restricciones[clase] = conecciones[index]
                
        #Guarda j en la clase
        try:
            soln[clase].append(index)
        except Exception:
            soln[clase] = [index]
        
    
    return soln

In [9]:
B = np.array([[0,0,0,1], [0,1,1,0], [0,0,0,0], [0,1,0,0]])
B

array([[0, 0, 0, 1],
       [0, 1, 1, 0],
       [0, 0, 0, 0],
       [0, 1, 0, 0]])

In [10]:
enumeracionDirigida(B)

{0: {1, 2, 3}, 1: {1, 2}, 2: set(), 3: {1, 2}}


{0: [0], 1: [1, 3], 2: [2]}